In [ ]:
import pandas as pd
from fuzzywuzzy import process, fuzz
import json
import uuid
import logging
import time

# ----------------- CONFIG ----------------- #
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

start_time = time.time()
logging.info("Starting product normalization process...")

# ----------------- LOAD MAPPINGS ----------------- #
def build_map(df, key_col="original_value", id_col="id"):
    return {str(k).strip().lower(): i for i, k in zip(df[id_col], df[key_col])}

brands = pd.read_json("elaf_brands.json")
categories = pd.read_json("elaf_categories.json")

brand_map = build_map(brands)
cat_map = build_map(categories)

def fuzzy_lookup(value, lookup_map, threshold=80, clean_chars=None):
    if not value or pd.isna(value):
        return None

    val = str(value).strip().lower()
    if clean_chars:
        for ch in clean_chars:
            val = val.replace(ch, " ")

    # Exact match
    if val in lookup_map:
        return lookup_map[val]

    # Fuzzy match
    keys = list(lookup_map.keys())
    keys_cleaned = keys
    if clean_chars:
        keys_cleaned = []
        for k in keys:
            val_cleaned = k.lower()
            for ch in clean_chars:
                val_cleaned = val_cleaned.replace(ch, " ")
            keys_cleaned.append(val_cleaned)

    match, score = process.extractOne(val, keys_cleaned, scorer=fuzz.token_sort_ratio)
    if score >= threshold:
        original_key = keys[keys_cleaned.index(match)]
        return lookup_map[original_key]
    return None

def get_first_nonnull(row, columns):
    for col in columns:
        if col in row and pd.notna(row[col]):
            return row[col]
    return None

def clean_sheet(df):
    df = df.dropna(axis=1, how="all")
    if df.columns.str.contains("Unnamed").all() or all([str(c) == "nan" for c in df.columns]):
        df.columns = df.iloc[0]
        df = df[1:]
    df.columns = df.columns.astype(str)
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
    return df

# ----------------- LOOKUP FUNCTIONS ----------------- #
def getBrandId(value, threshold=80):
    return fuzzy_lookup(value, brand_map, threshold)

def getCategoryId(row, threshold=80):
    category_columns = [
        "التصنيف الفرعي الثالث", 
        "التصنيف الفرعي الثاني", 
        "الصنف الثاني", 
        "التصنيف الفرعي الأول", 
        "الصنف الاول"
    ]
    val = get_first_nonnull(row, category_columns)
    return fuzzy_lookup(val, cat_map, threshold, clean_chars=[",", "-"])

# ----------------- PROCESS EXCEL ----------------- #
df_sheets = pd.read_excel("ElafProducts.xlsx", sheet_name=None)
mainCats = [sheet for sheet in df_sheets.keys() if sheet not in ["Countries Code", "طعام طازجold"]]

all_products = []

for sheet_name in mainCats:
    logging.info(f"Processing sheet: {sheet_name}")
    df = clean_sheet(df_sheets[sheet_name])

    for _, row in df.iterrows():
        name_ar = get_first_nonnull(row, ["اسم المنتج بالعربي", "اسم المنتج (عربي)"])
        name_en = get_first_nonnull(row, ["اسم المنتج بالنجليزي", "اسم المنتج (إنجليزي)"])
        if not name_ar or not name_en:
            continue

        cat = getCategoryId(row)
        if not cat:
            cat = fuzzy_lookup(sheet_name, cat_map, threshold=70, clean_chars=[",", "-"])

        product = {
            "id": str(uuid.uuid()),
            "categoryId": cat,
            "brandId": getBrandId(get_first_nonnull(row, ["العلامه التجاريه", "العلامة التجارية"])),
            "name": {"ar": name_ar, "en": name_en},
            "qty": get_first_nonnull(row, ["العدد", "العدد بدون (pcs)"]),
            "uom": get_first_nonnull(row, ["وحدة القياس"]),
            "size": get_first_nonnull(row, ["السعة او الحجم", "السعه"]),
            "price": get_first_nonnull(row, ["السعر بالريال السعودي", "السعر  بالريال السعودي"])
        }

        all_products.append(product)

    logging.info(f"Finished sheet: {sheet_name}, processed {len(df)} rows")

# ----------------- EXPORT ----------------- #
with open("elaf_products.json", "w", encoding="utf-8") as f:
    json.dump(all_products, f, ensure_ascii=False, indent=4, default=str)

total_time = time.time() - start_time
logging.info(f"Total products processed: {len(all_products)}")
logging.info(f"Total time: {total_time:.2f} seconds")


2025-12-09 01:34:41 [INFO] Starting product normalization process...
2025-12-09 01:34:44 [INFO] Processing sheet:  طعام طازج
2025-12-09 01:34:44 [WARNING] Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']
2025-12-09 01:34:44 [INFO] Finished sheet:  طعام طازج, processed 1639 rows
2025-12-09 01:34:44 [INFO] Processing sheet: أكسسوارات السيارات
2025-12-09 01:34:44 [INFO] Finished sheet: أكسسوارات السيارات, processed 2446 rows
2025-12-09 01:34:44 [INFO] Processing sheet: خزانة الطعام
2025-12-09 01:34:44 [INFO] Finished sheet: خزانة الطعام, processed 5960 rows
2025-12-09 01:34:44 [INFO] Processing sheet: المشروبات
2025-12-09 01:34:44 [INFO] Finished sheet: المشروبات, processed 1642 rows
2025-12-09 01:34:44 [INFO] Processing sheet: خضار و فواكه
2025-12-09 01:34:44 [INFO] Finished sheet: خضار و فواكه, processed 425 rows
2025-12-09 01:34:44 [INFO] Processing sheet: العاب و مستلزمات النشاطات الخار
2025-12-09 01:34:44 [INFO] Finished sheet: الع